In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
N=20
layers=5

In [3]:
def Ising_Hamiltonian_qml(J,g,N):
    '''
    Ising Hamiltonian with a transverse field (pennylane notation):
    H = J * sum_{i,j} Z^i Z^(i+1) + g * sum_i X^i
    ...
    Parameters
    -------
    J : float
        Strenght of the interaction.
    g : float
        Strenght of the transverse magnetic field.
    '''
    
    H = g*qml.PauliZ(N-1) 
    for i in range(0,N-1):
        H = H+ g*qml.PauliZ(i)+J*qml.PauliX(i)@qml.PauliX(i+1)
    #H = H + g*qml.PauliX(N-1) + J*qml.PauliZ(N-1)@qml.PauliZ(0)

    return H

In [4]:
H = Ising_Hamiltonian_qml(-1,1,N)
dev = qml.device('lightning.qubit', wires=N)
@qml.qnode(dev,diff_method='adjoint')
def cost(params):
    for layer in range(layers):
        for i in range(N):
            qml.RY(params[2*layer*N+i],i)
        for i in range(int(N/2)):
            qml.CZ(wires=[2*i,2*i+1])
        for i in range(N):
            qml.RY(params[2*layer*N+N+i],i)
        for i in range(1,int(N/2)):
            qml.CZ(wires=[2*i-1,2*i])
        qml.CZ(wires=[N-1,0])
        qml.Barrier(range(0,N))
    qml.Barrier(range(0,N))
    return qml.expval(H)

In [7]:
params = 0.01*np.random.random(2*N*layers,requires_grad=True)

In [8]:
cost(params)

array(19.94941532)